In [1]:
import pandas as pd
wallet_df = pd.read_csv('Wallets.csv')  
wallet_addresses = wallet_df['wallet_id'].tolist()
print("Total wallets loaded:", len(wallet_addresses))
wallet_df

Total wallets loaded: 103


,wallet_id
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0x0795732aacc448030ef374374eaae57d2965c16c
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
...,...
98,0xf60304b534f74977e159b2e159e135475c245526
99,0xf67e8e5805835465f7eba988259db882ab726800
100,0xf7aa5d0752cfcd41b0a5945867d619a80c405e52
101,0xf80a8b9cfff0febf49914c269fb8aead4a22f847


In [2]:
import requests
from tqdm import tqdm
def fetch_wallet_data(wallet):
    return {
        "supplied_amount": 5000,
        "borrowed_amount": 3000,
        "repaid_amount": 2000,
        "liquidations": 1,
        "health_factor": 1.1  
    }

wallet_features = []

for wallet in tqdm(wallet_addresses[:100]): 
    data = fetch_wallet_data(wallet)
    data['wallet_id'] = wallet
    wallet_features.append(data)

df = pd.DataFrame(wallet_features)
df.head()


100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<?, ?it/s]


,supplied_amount,borrowed_amount,repaid_amount,liquidations,health_factor,wallet_id
0,5000,3000,2000,1,1.1,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,5000,3000,2000,1,1.1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,5000,3000,2000,1,1.1,0x0795732aacc448030ef374374eaae57d2965c16c
3,5000,3000,2000,1,1.1,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,5000,3000,2000,1,1.1,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


In [3]:
df['borrow_to_supply_ratio'] = df['borrowed_amount'] / (df['supplied_amount'] + 1)
df['repaid_to_borrow_ratio'] = df['repaid_amount'] / (df['borrowed_amount'] + 1)
features = ['borrow_to_supply_ratio', 'repaid_to_borrow_ratio', 'liquidations', 'health_factor']
df.fillna(0, inplace=True)

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df[features]), columns=features)
df_normalized = pd.concat([df[['wallet_id']], df_scaled], axis=1)
df_normalized.head()

,wallet_id,borrow_to_supply_ratio,repaid_to_borrow_ratio,liquidations,health_factor
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0.0,0.0,0.0,0.0
1,0x06b51c6882b27cb05e712185531c1f74996dd988,0.0,0.0,0.0,0.0
2,0x0795732aacc448030ef374374eaae57d2965c16c,0.0,0.0,0.0,0.0
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,0.0,0.0,0.0,0.0
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,0.0,0.0,0.0,0.0


In [5]:
weights = {
    'borrow_to_supply_ratio': 0.3,
    'repaid_to_borrow_ratio': 0.3,
    'liquidations': 0.2,
    'health_factor': 0.2
}

df_normalized['score'] = (
    df_normalized['borrow_to_supply_ratio'] * weights['borrow_to_supply_ratio'] +
    df_normalized['repaid_to_borrow_ratio'] * weights['repaid_to_borrow_ratio'] +
    (1 - df_normalized['liquidations']) * weights['liquidations'] +  # inverse risk
    df_normalized['health_factor'] * weights['health_factor']
)

df_normalized['score'] = (df_normalized['score'] * 1000).round().astype(int)
df_final = df_normalized[['wallet_id', 'score']]
df_final.head()


,wallet_id,score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,200
1,0x06b51c6882b27cb05e712185531c1f74996dd988,200
2,0x0795732aacc448030ef374374eaae57d2965c16c,200
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,200
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,200


In [6]:
df_final.to_csv("wallet_risk_scores.csv", index=False)
print("Saved to wallet_risk_scores.csv")

Saved to wallet_risk_scores.csv
